<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Tuesday 31 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 2: Bayesian inference</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>2 </sup>Ioannis Bistinas, 
<sup>3, 4 </sup>Chantelle Burton, 
<sup>1 </sup>Tobias Marthews, 
<sup>5 </sup>Rhys Whitley
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Vrije Universiteit Amsterdam, Faculty of Earth and Life Sciences, Amsterdam, Netherlands
<br>
<sup>3 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>4 </sup>Geography, University of Exeter, Exeter, United Kingdom
<br>
<sup>5 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to quantify the model parameters of a global fire model (defined below). The model is driven by a number of covariates (X<sub>i=1, 2, ... M</sub>) that describe: cropland, pasture and urban area footprints; frequency of lightening ignitions, population density, net primary productivity (NPP) and <i>Alpha</i>, a proxy measure of available soil moisture in the root zone. The model attempts to predict the impact of fire through burnt area and is thus the model target (Y).
</p>
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
</font>
</center>
<hr>
</div>

### Model description
The model considers percentage of burnt area to be the joint product of a set of conditions that modulate fire through fuel load, ignitions, moisture and supression. Each function assumes some equilibrium point that desribes the optimal conditions for fire, that may be proportionally modified through some empirical relationship. These are briefly outlined below for the sake of comprehension in this notebook, but can be referred to in more detail in the model protocol located in the docs/ folder (<a href='file:///localhost/../docs/Model_description.pdf'>model protocol</a>).

\begin{eqnarray}
    F_{burn} &=& \prod^{4}_{i=1}f_i(x) \\[1em]
    f_1 &=& f(x_{fuel}) \\[0.5em]
    f_2 &=& f(x_{moist}) \\[0.5em]
    f_3 &=& 1 - f(x_{ignite}) \\[0.5em]
    f_4 &=& 1 - f(x_{suppr})
\end{eqnarray}

Each of the above 4 functions describe some measure fire conditions, and are described by a sigmoid:

\begin{equation}
    f(x_{i=1\ldots 4}) = \frac{1}{1 + \exp\{-k_0\cdot(x_i-x_0)\}}
\end{equation}

Three of the four condition functions describe an aggregation of other climate and land-use covariates; the fuel function considers only NPP and is directly modified by changes in these values. Consequently for functions $f_{i=2\ldots4}$, the parameter $x_i$ is an aggregation of independent drivers; these are now described below.  

#### Fuel load covariate
\begin{equation}
    x_{fuel} = NPP 
\end{equation}

#### Moisture covariate
\begin{equation}
    x_{moist} = \frac{\alpha + k_p\cdot p_{atm}}{1 + k_p}
\end{equation}

#### Ignition covariate 
\begin{equation}
    x_{ignite} = F_L\cdot\min(1.0,\space 0.0408\cdot F_L^{-0.4180}) + k_p\cdot A_{pasture} + k_{d1}\cdot\rho_{population}
\end{equation}

#### Supression covariate 
\begin{equation}
    x_{supress} = A_{urban} + k_C\cdot A_{Crop} + k_{d2}\cdot\rho_{population} 
\end{equation}

This leaves 11 free parameters that need to be optimised against observations of burnt area.  

### Load libraries

In [77]:
import os
import numpy as np
import pandas as pd

import pymc3 as pm3 
import pymc as pm2
from theano import tensor as tt

ImportError: dlopen(/usr/local/lib/python3.5/site-packages/pymc/flib.cpython-35m-darwin.so, 2): Library not loaded: /usr/local/opt/gcc/lib/gcc/5/libgfortran.3.dylib
  Referenced from: /usr/local/lib/python3.5/site-packages/pymc/flib.cpython-35m-darwin.so
  Reason: image not found

## 2.1 Fire limitation model definition

Could possibly contain this in a class object, but I'm not sure theano can instantiate the object to be used by the GPU. If I've made absolutely no sense just then, then I would leave the following as is.

In [73]:
def lightning(flash):
    """
    Transform flashes to some lightning measure 
    """
    return flash*min(1.0, 0.0408*flash**(-0.4180))

def fuel_load(NPP):
    """
    Definition to describe fuel load: while return the input; capability to be modified later.
    """
    return NPP

def moisture(alpha, p_atm=0, kp=0):
    """
    Definition to describe moisture: currently equals alpha; capability to be modified later.
    """
    return (alpha + kp*p_atm)/(1.0 + kp) 

def ignition(lightning, pasture_area, pop_density, kp, kd1):
    """
    Definition for the measure of ignition
    """
    return lightning + kp*pasture_area + kd1*pop_density

def supression(urban_area, crop_area, pop_density, kc, kd2):
    """
    Definition for the measure of fire supression
    """
    return urban_area + kc*crop_area + kd2*pop_density

def tt_sigmoid(x, a, b):
    """
    Sigmoid function to describe limitation using tensor
    """
    #return 1.0/(1.0 - tt.exp(-b*(x - a)))
    return 1.0/(1.0 - np.exp(-b*(x - a)))
 
def fire_limitation0(x_fload, x_moist, x_ignite, x_suppr, a_v, b_v):
    """
    Full fire limiation definition
    """
    f1 = tt_sigmoid(x_fload, a_v[0], b_v[0])
    f2 = tt_sigmoid(x_moist, a_v[1], b_v[1])
    f3 = 1.0 - tt_sigmoid(x_ignite, a_v[2], b_v[2])
    f4 = 1.0 - tt_sigmoid(x_suppr, a_v[3], b_v[3])
    
    return  f1*f1*f3*f4

def fire_limitation(x_v, a_v, b_v):
    """
    Full fire limiation definition
    """
    fun_i = [tt_sigmoid(x, a, b) \
            if i < 2 else \
            1 - tt_sigmoid(x, a, b) \
            for (i, (x, a, b)) in enumerate(zip(x_v, a_v, b_v))]
    
    # return the product of the functions
    return np.prod(fun_i)

## 2.2 Import data

Load data and do any necessary transformation needed for the Bayesian modelling framework

In [20]:
DATAPATH = os.path.expanduser("../data/globfire.csv.gz")

fd = pd.read_csv(DATAPATH)

Transform the flash measures to a measure of lightning ignition. Probably better to redo column names in previous notebook to save confusion.

In [44]:
fd['x_light'] = [lightning(x) if x > 0 else 0 for x in fd['lightning_ignitions'].values]

Sanity check...

In [45]:
fd.head()

,alpha,cropland,fire,lightning_ignitions,NPP,pasture,population_density,urban_area,x_light
0,0.469404,0.0,0.0,0.0,121.678574,0.0,0.0,0.0,0.0
1,0.549735,0.0,0.0,0.0,112.512817,0.0,0.0,0.0,0.0
2,0.701645,0.0,0.0,0.0,109.520493,0.0,0.0,0.0,0.0
3,1.243935,0.0,0.0,0.0,149.522217,0.0,0.0,0.0,0.0
4,1.243933,0.0,0.0,0.0,51.428570,0.0,0.0,0.0,0.0


## 2.3 Baysian framework

### 2.3.1 Prior definitions
Because I have no idea what the uncertainty on these parameters should look like, I've set them all as uniform. Some of them can logically be describe as exponential or half-normal as they can't physically go below zero, but we can play around with that later.

`pymc3` is quite funky in that it allows me to create an empty `Model()` object and just add things to it as I need them using a `with` statement. I've called our Bayesian model `fire_error` as that is what we are trying to Quantify.

In [50]:
# first for the sigmoids (the 4's at the end state that I want 4 of them)
with pm3.Model() as fire_error:
    a_s4 = pm3.Uniform('sig_pos', -2, 2, 4)
    b_s4 = pm3.Uniform('sig_slope', 0, 5, 4)
    
# now for the hyper-parameters that describe the independent fire condition covariates
with fire_error:
    kp = pm3.Uniform('kp', -100, 100)
    kc = pm3.Uniform('kc', -100, 100)
    kd1 = pm3.Uniform('kd1', -100, 100)
    kd2 = pm3.Uniform('kd2', -100, 100)
    
# describe the standard deviation in the error term
with fire_error:
    sigma = Uniform('sigma', 0, 20)

### 2.3.2 Likelihood definition

For the sake of simplicity (and because I don't really know any better), we define the model error as normally distributed (i.i.d.) although it most likely isn't. We could make this more complicated later by defining the error as heteroscedastic, but I wouldn't bother with that until we have some idea of the convergence.

In [49]:
def minimum_error(y_obs, y_pred):
    return np.sum(np.abs(yobs - y_pred))

In [74]:
with fire_error:
    
    # determine the composite covariate functions
    f_1 = fd["NPP"].values
    f_2 = fd["alpha"].values
    
    f_3 = ignition(fd["x_light"].values, \
                   fd["pasture"].values, \
                   fd["population_density"].values, \
                   kp, kd1)
    
    f_4 = supression(fd["urban_area"].values, \
                     fd["cropland"].values, \
                     fd["population_density"].values, \
                     kc, kd2)
    
    # make a prediction of fire limitation based on the current state
    prediction = fire_limitation([f_1, f_2, f_3, f_4], a_s4, b_s4) 
    #prediction = fire_limitation0(f_1, f_2, f_3, f_4, a_s4, b_s4) 
    
    # calculate the error between observed and predicted 
    error = pm3.Normal('error', mu=prediction, sd=sigma, observed=fd['fire'].values)

TypeError: TensorType does not support iteration. Maybe you are using builtin.sum instead of theano.tensor.sum? (Maybe .max?)

----

In [ ]:
a_s4 = pm.Uniform('sig_pos', -2, 2, 4)
b_s4 = pm.Uniform('sig_slope', 0, 5, 4)

kp = pm.Uniform('kp', -100, 100)
kc = pm.Uniform('kc', -100, 100)
kd1 = pm.Uniform('kd1', -100, 100)
kd2 = pm.Uniform('kd2', -100, 100)

sigma = Uniform('sigma', 0, 20)